# Pipeline para a 3W

Neste notebook será implementado uma Pipiline de ML aplicado ao problema da dataset 3W da Petrobras.

Para tal será usada a biblioteca TensorFlow Extended.

Autoria: Marcus Carr

### Nomenclatura

instance/instância de um **evento**: equivale a 1 arquivo .csv

**sample**: cada timestep dentro de um .csv

### Estrutura do projeto. 

Como ainda não sei como será tudo com a implementação do módulos do TFX, vou deixar um módulo principal por enquanto.

Posteriormente, dá para analisar se seria mais adequado quebrar em diferentes módulos a estrutra do código.

In [10]:
# Verificar que TFX está instalado
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

2023-08-24 01:07:31.565637: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-24 01:07:31.574088: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 01:07:31.667386: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-24 01:07:31.669096: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-24 01:07:33.036358: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

TensorFlow version: 2.12.1
TFX version: 1.13.0


### Seting up variables

These will define our pipeline.

In [11]:
import raw_data_acquisition as rda
import raw_data_inspector as rdi
from constants import utils, config
import models

# Set default logging level.
from absl import logging
logging.set_verbosity(logging.DEBUG)

Adquirir dados!

In [12]:
rda.acquire_dataset_if_needed() # 17min48s (local) -> 1m55s (server)

INFO:absl:Directory with the biggest version: /home/ubuntu/lemi_3w/data/dataset_converted_v10101
INFO:absl:Version: 10101
INFO:absl:Latest local version is 10101
INFO:absl:Going to fetch config file from $https://raw.githubusercontent.com/petrobras/3W/main/dataset/dataset.ini
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2786  100  2786    0     0   4623      0 --:--:-- --:--:-- --:--:--  4627
INFO:absl:Latest online version is 10101
INFO:absl:Found existing converted data with dataset version of 10101


In [13]:
latest_converted_data_path = rda.get_latest_local_converted_data_version(config.DIR_PROJECT_DATA)[0]
print(f"Size of directory with converted files is: {utils.get_directory_size(latest_converted_data_path)/(1024**3):.3f} GB")

INFO:absl:Directory with the biggest version: /home/ubuntu/lemi_3w/data/dataset_converted_v10101
INFO:absl:Version: 10101


Size of directory with converted files is: 1.263 GB


In [14]:
inspector = rdi.RawDataInspector(
    latest_converted_data_path,
    config.PATH_DATA_INSPECTOR_CACHE,
    True
)

inspector.get_metadata_table()

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
74203bb,NORMAL,REAL,1.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-24 03:00:00,491413,17885
9fbd6f9,NORMAL,REAL,2.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-09 06:00:00,520149,17933
28804c5,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-08 09:00:31,349159,17970
42afe91,NORMAL,REAL,8.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-07-01 14:01:35,251878,17799
fa71d94,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-23 19:00:00,279736,17949
...,...,...,...,...,...,...,...
ea66cf6,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2315902,61999
34f032a,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2259537,61999
876a969,SEVERE_SLUGGING,REAL,14.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-09-25 06:00:42,1005714,17959


In [15]:
inspector.get_metadata_table()['well_id']

hash_id
74203bb     1.0
9fbd6f9     2.0
28804c5     6.0
42afe91     8.0
fa71d94     6.0
           ... 
ea66cf6     NaN
34f032a     NaN
876a969    14.0
deac7ec     NaN
c8867fa     NaN
Name: well_id, Length: 1978, dtype: float64

In [16]:
#inspector.get_metadata_table(class_types=[models.EventClassType(8).name])
inspector.get_metadata_table()

,class_type,source,well_id,path,timestamp,file_size,num_timesteps
hash_id,,,,,,,
74203bb,NORMAL,REAL,1.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-24 03:00:00,491413,17885
9fbd6f9,NORMAL,REAL,2.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-09 06:00:00,520149,17933
28804c5,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-05-08 09:00:31,349159,17970
42afe91,NORMAL,REAL,8.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-07-01 14:01:35,251878,17799
fa71d94,NORMAL,REAL,6.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-08-23 19:00:00,279736,17949
...,...,...,...,...,...,...,...
ea66cf6,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2315902,61999
34f032a,SEVERE_SLUGGING,SIMULATED,NaN,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,NaT,2259537,61999
876a969,SEVERE_SLUGGING,REAL,14.0,/home/ubuntu/lemi_3w/data/dataset_converted_v1...,2017-09-25 06:00:42,1005714,17959


In [17]:
from sklearn.model_selection import train_test_split
import pandas as pd


y = inspector.get_metadata_table()
X_mock = pd.DataFrame(0, index=range(y.shape[0]), columns=range(7))

print(X_mock.shape)
print(y.shape)

x_train, x_test, y_train, y_test = train_test_split(y, y,
                                                    test_size=0.25,
                                                    random_state=0,
                                                    stratify=y[['source', 'class_type']])
                                            
                                                    

(1978, 7)
(1978, 7)


In [18]:
from raw_data_manager import models

def get_table_by_anomaly_source(data_all: pd.DataFrame) -> pd.DataFrame:
    anomaly = []
    real_count = []
    simul_count = []
    drawn_count = []
    soma = []

    for anomaly_type in models.EventClassType:
        anomaly.append(anomaly_type.name)
        
        real_count.append(
            len(data_all[(data_all['class_type'] == anomaly_type.name) & (data_all['source'] == 'REAL')]))
        simul_count.append(
            len(data_all[(data_all['class_type'] == anomaly_type.name) & (data_all['source'] == 'SIMULATED')]))
        drawn_count.append(
            len(data_all[(data_all['class_type'] == anomaly_type.name) & (data_all['source'] == 'HAND_DRAWN')]))
        soma.append(
            len(data_all[data_all['class_type'] == anomaly_type.name]))

    anomaly.append('Total')
    real_count.append(sum(real_count))
    simul_count.append(sum(simul_count))
    drawn_count.append(sum(drawn_count))
    soma.append(sum(soma))

    data = {
        'anomaly': anomaly,
        'real_count': real_count,
        'simul_count': simul_count,
        'drawn_count': drawn_count,
        'soma' : soma,
    }

    # Create the DataFrame
    df_source = pd.DataFrame(data)
    df_source.set_index('anomaly', inplace=True)
    return df_source

In [19]:
get_table_by_anomaly_source(x_train)

,real_count,simul_count,drawn_count,soma
anomaly,,,,
NORMAL,445,0,0,445
ABRUPT_INCREASE_BSW,4,85,8,97
SPURIOUS_CLOSURE_DHSV,16,12,0,28
SEVERE_SLUGGING,24,55,0,79
FLOW_INSTABILITY,258,0,0,258
RAPID_PRODUCTIVITY_LOSS,8,329,0,337
QUICK_RESTRICTION_PCK,5,161,0,166
SCALING_IN_PCK,4,0,8,12
HYDRATE_IN_PRODUCTION_LINE,0,61,0,61


In [20]:
get_table_by_anomaly_source(x_test)

,real_count,simul_count,drawn_count,soma
anomaly,,,,
NORMAL,149,0,0,149
ABRUPT_INCREASE_BSW,1,29,2,32
SPURIOUS_CLOSURE_DHSV,6,4,0,10
SEVERE_SLUGGING,8,19,0,27
FLOW_INSTABILITY,86,0,0,86
RAPID_PRODUCTIVITY_LOSS,3,110,0,113
QUICK_RESTRICTION_PCK,1,54,0,55
SCALING_IN_PCK,1,0,2,3
HYDRATE_IN_PRODUCTION_LINE,0,20,0,20


In [21]:
x_test['path'].tolist()

['/home/ubuntu/lemi_3w/data/dataset_converted_v10101/4/WELL-00010_20180416080150.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/4/WELL-00005_20170624160124.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/2/SIMULATED_00006.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/3/WELL-00014_20170918120103.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/4/WELL-00002_20140109050025.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/2/WELL-00011_20140720120102.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/0/WELL-00002_20170623050021.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/0/WELL-00008_20170819070246.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/3/WELL-00014_20170918020114.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/0/WELL-00002_20170805020000.parquet',
 '/home/ubuntu/lemi_3w/data/dataset_converted_v10101/0/WELL-00006_20170827030218.parquet',
 '/home/u

In [22]:
import models
list = [models.EventClassType.ABRUPT_INCREASE_BSW, models.EventClassType.FLOW_INSTABILITY]
numbers = [class_type.value for class_type in list]

"-".join(map(str, numbers))

'1-4'

In [23]:
len(models.EventClassType)

9